In [1]:
import pandas as pd
from sqlalchemy import create_engine
import yaml
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
config = yaml.safe_load(open('dart_config.yml'))
db_url = 'mysql://' + config['DB_USER'] + ':' + config['DB_USER_PASSWORD'] + '@' + config['DB_HOST'] \
                            + '/' + config['DB_NAME'] + '?charset=utf8'
engine = create_engine(db_url, encoding='utf-8')
conn = engine.connect()
company_list = pd.read_sql_table(config['DB_TABLE_NAME_1'], con=conn,
                          columns=['name'])['name']
company_list = company_list.drop_duplicates().reset_index(drop=True)
company_list.head()

0    롯데케미칼
1     대한유화
2     이화산업
3    디케이앤디
4     송원산업
Name: name, dtype: object

In [3]:
# Read csv
trade_df = pd.read_csv("./data/trade.csv", encoding="utf-8", parse_dates=['ISSUEDT'],
                       dtype={'HSCD': str, "QTY": float, 'BYRADDR2': str, 'SPLYADDR2': str})
trade_df.head()

,HSCD,QTY,QTYUNIT,AMT,CUR,USDUNTPRICE,UNTPRICE,ITEM,BYRORGNM1,BYRPRTNUM,...,BYRADDR2,SPLYORGNM1,SPLYPRTNUM,SPLYADDR1,SPLYADDR2,ISSUEDT,TOTQTY,TOTQTYUNIT,TOTAMT,TOTAMTCUR
0,7210491090,84000.0,KG,45360.00,USD,540.00,540.00,GI COIL,주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
1,7210491090,1463000.0,KG,821355.00,USD,585.00,585.00,GI COIL,주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
2,7210491090,600000.0,KG,398580.00,USD,664.30,664.30,HG COIL - POSCO,주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
3,7225509000,2398000.0,KG,416301.78,USD,501.15,501.15,CR COIL (0.5MM<=T<1.0MM),주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
4,7209169000,23100000.0,KG,8134262.00,USD,490.00,490.00,CR COIL (3MM<=T),주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD


In [4]:
buyer_list = trade_df['BYRORGNM1']
buyer_list.head()

0    주식회사 포스코대우
1    주식회사 포스코대우
2    주식회사 포스코대우
3    주식회사 포스코대우
4    주식회사 포스코대우
Name: BYRORGNM1, dtype: object

In [5]:
supplier_list = trade_df['SPLYORGNM1']
supplier_list.head()

0    POSCO
1    POSCO
2    POSCO
3    POSCO
4    POSCO
Name: SPLYORGNM1, dtype: object

In [6]:
merge_list = buyer_list.append(supplier_list, ignore_index=True).drop_duplicates().reset_index(drop=True)
merge_list.head()

0         주식회사 포스코대우
1        도레이케미칼 주식회사
2    도레이첨단소재주식회사 4공장
3        도레이첨단소재주식회사
4    도레이첨단소재주식회사 2공장
dtype: object

In [7]:
parsed_merge_list = merge_list.str.replace("주식회사", "").str.replace("\(주\)", "")\
        .str.replace("\(유\)", "").str.replace("㈜", "").str.replace(" ", "")
parsed_merge_list.head()

0         포스코대우
1        도레이케미칼
2    도레이첨단소재4공장
3       도레이첨단소재
4    도레이첨단소재2공장
dtype: object

In [8]:
original_merge_list = company_list.append(merge_list, ignore_index=True)
parsed_merge_list = company_list.append(merge_list, ignore_index=True)
del merge_list
print('original_merge_list length:', len(original_merge_list), 'parsed_merge_list length:', len(parsed_merge_list))

original_merge_list length: 54100 parsed_merge_list length: 54100


In [9]:
# make tfidf matrix set
tfidf = TfidfVectorizer(analyzer='char', ngram_range=(2,3))
tfidf_matrix = tfidf.fit_transform(parsed_merge_list)
tfidf_matrix.shape

(54100, 107373)

In [10]:
print('total company count:', len(company_list))

total company count: 796


## Make dictionary

In [178]:
company_dict = {}

In [2371]:
# Set company for comparing
company_i += 1
cosine_sim = linear_kernel(tfidf_matrix[company_i], tfidf_matrix)
# print(company_i, original_merge_list[company_i])

# Result comparing
sorted_index = sorted(range(len(cosine_sim[0])), key=lambda i: cosine_sim[0][i])[-10:]
sorted_index.reverse()
for i, v in enumerate(sorted_index):
    print(str(i), str(v), original_merge_list[v], str(cosine_sim[0][v]))
#    if (cosine_sim[0][v] < 0.4):
#        break

0 521 네오티스 0.9999999999999999
1 31057 (주)네오티스 0.8282468078644203
2 37697 오티스엘리베이터유한회사 0.31013196234975804
3 31105 지오티 0.21515661943694672
4 22409 오티티 0.20373361153128883
5 12229 엠오티 0.196735849669779
6 9891 주) 오티 0.17863409667128302
7 41439 네오텍 0.17746800706916777
8 21541 유티스(주) 0.16924039452366318
9 16635 (주)티스코 0.16897800244989655


In [2372]:
sim_list = []
for i, v in enumerate(sorted_index):
    sim_list.append(original_merge_list[v])
    if (i == 1):
        break
for v, i in enumerate(sim_list):
    print(v, i)

0 네오티스
1 (주)네오티스


In [2373]:
company_dict[original_merge_list[company_i]] = sim_list
print(original_merge_list[company_i], "added to dictionay !!!")
# company_dict

네오티스 added to dictionay !!!


### Warning!!

In [2369]:
sim_list.pop(2)
for v, i in enumerate(sim_list):
    print(v, i)

0 신한
1 신한
2 (유)신한


## Save

In [1929]:
import json
with open('data/company.json', 'w') as f:
    json.dump(company_dict, f)

## Open

In [240]:
import json
with open('data/company.json') as f:
    my_dict = json.load(f)
my_dict

{'국도화학': ['국도화학', '국도화학(주)', '국도화학주식회사', '국도화학 주식회사', '국도화학(주) 익산지점'],
 '금호석유화학': ['금호석유화학',
  '금호석유화학',
  '금호석유화',
  '금호석유화학(주)',
  '금호석유화학㈜',
  '주금호석유화학',
  '금호석유화학주식회사',
  '금호석유화학 (주)',
  '금호석유(주)',
  '금호석유화학(주)울산고무공장',
  '금호석유화학(주)여수정밀화학공장',
  '금호석유화학(주)여수고무제1공장'],
 '대주전자재료': ['대주전자재료', '대주전자재료', '대주전자재료(주)', '대주전자재료주식회사'],
 '대한유화': ['대한유화', '대한유화', '대한유화(주)', '대한유화주식회사', '대한유화공업(주)', '대한유압'],
 '디케이앤디': ['디케이앤디', '디케이앤디', '(주)디케이앤디', '주식회사 디케이앤디'],
 '롯데케미칼': ['롯데케미칼',
  '롯데케미칼',
  '롯데케미칼(주)',
  '롯데케미칼주식회사',
  '롯데케미칼 (주)',
  '롯데케미칼 주식회사',
  '(주)롯데케미칼',
  '롯데케미컬(주)',
  '롯데케미컬 주식회사'],
 '멜파스': ['멜파스', '주)멜파스', '(주)멜파스', '주식회사 멜파스'],
 '미원상사': ['미원상사', '미원상사(주)', '미원상사 주식회사', '미원상사 (주)', '미원상사㈜', '(주)미원상사'],
 '송원산업': ['송원산업', '송원산업', '송원산업(주)', '송원산업주식회사', '송원산업 (주)', '송원산업 주식회사'],
 '이화산업': ['이화산업', '이화산업', '이화산업사', '이화산업(주)'],
 '일진머티리얼즈': ['일진머티리얼즈',
  '일진머티리얼즈',
  '일진머티리얼즈(주)',
  '일진머티리얼즈 주식회사',
  '일진머티리얼즈 (주)',
  '일진머티리얼즈(주)익산공장'],
 '한국쉘석유': ['한국쉘석유',
  '한국쉘석유',
  '한국쉘석유주식회사',
  '한국쉘석

## TEST

In [126]:
trade_df.loc[trade_df['BYRORGNM1'] == '현대자동차']['HSCD'].drop_duplicates()

792170     8708999000
792177     8302300000
792209     7318220000
792510     9031909000
792518     8512201090
792519     3923500000
792520     8708991030
792548     8543709090
792553     8708309000
792593     9030339000
792725     8708400000
792849     8301200000
792857     8302100000
792925     8501101000
792931     8501311010
792934     8708290000
792957     3926300000
792962     8301600000
793240     7326909000
793271     4009310000
793282     4009420000
793284     8708930000
793295     4016930000
793299     4009110000
793322     4016991090
793326     8415900000
793386     4009120000
793440     4009410000
793498     9401902000
793755     8536502000
              ...    
889182     9027509000
889261     8536490000
892477     3921901000
892507     4415100000
892619     4415200000
892932     8207909000
894509     3402901000
894510     3402903000
895094     8414901000
895207     8708502000
895654     8512202090
896409     8431100000
903947     3918109000
904261     3209101019
904266    

In [127]:
trade_df.loc[trade_df['HSCD'] == '3923500000']['BYRORGNM1'].drop_duplicates()

47242              주식회사 포스코대우
53902              엘지이노텍 주식회사
132944             (주)지앤지트레이딩
331914                제라 주식회사
336422                  (주)사람
342035                (주)네트코스
342062               유앤팩 주식회사
342191                 (주)파세코
344238                  (주)신영
346942                건식무역(주)
360620                (주)세신산업
370437             한국후지제록스(주)
372635              (주)신흥문구공업
384526              고려인삼과학(주)
461201                  기아자동차
699674             현대다이모스주식회사
699873               한국스미토모상사
705920           (주)에이테크오토모티브
722031           뉴팩코퍼레이션 유한회사
725462               한국지엠주식회사
772986             (주)트라이던트상사
780586                 (주)코팩코
792519                  현대자동차
827410              주식회사 쎄라테크
887840            (주)피오라뷰티코리아
910290               에스케이이(주)
917380             주식회사세광하이테크
930540               (주)톨리코리아
930572          이피에스에스글로벌주식회사
931306     애디언트오토모티브인테리어코리아(주
                  ...        
1140323               오리엔탈F&B
1142064             혜성음료 주식회사
1143061   

In [175]:
for i, v in enumerate(company_list):
    if '삼성' in v:
        print(i, v)

In [39]:
company_i = 2
cosine_sim = linear_kernel(test_matrix[company_i], tfidf_matrix)
print(test_name[company_i])

dk&d


In [40]:
sorted_index = sorted(range(len(cosine_sim[0])), key=lambda i: cosine_sim[0][i])[-100:]
sorted_index.reverse()
for i, v in enumerate(sorted_index):
    print(str(i), str(v), csv_company_series[v], str(cosine_sim[0][v]))

0 69325 주식회사 디케이앤디 (DK&D CO.LTD) 0.4857910369292537
1 68821 주식회사 디케이앤디(DK&D Co.Ltd) 0.4857910369292537
2 59802 주식회사 디케이앤디(DK&D CO.Ltd) 0.4857910369292537
3 46092 K&C 0.2529445494267581
4 4030 K&J 0.23562164160311044
5 48604 주식회사 DKS 0.1993078875100243
6 97659 DKR 0.19073495492699127
7 26947 DKCS 0.16034756490524588
8 83710 K&B상사 0.15988331991986116
9 69913 K & B상사 0.15988331991986116
10 53193 DK상사 0.15365185711802878
11 13329 DK섬유 0.14710257071660907
12 18538 동신E&D 0.14653898290149303
13 54742 DK동신주식회사 0.14363428710612208
14 9307 DK글로벌 0.14361452036587688
15 53789 신흥F&D 0.14264290404957086
16 37092 (유)성지F&D 0.1421697633855379
17 69694 (주)대도F&D 0.14195148542072344
18 14655 DK 강업  0.14087353496409583
19 71145 청해S&D 0.13969621625075196
20 69335 평화 M&D 0.13939114531895824
21 18201 가람D&D 0.13865147326419763
22 93943 한림 W&D 0.13792107972818643
23 81165 한림W&D 0.13792107972818643
24 54923 dk메디칼 0.13529022340477692
25 788 DK솔루션 0.13488729211184328
26 68653 엠제이R&D 0.13416354598019967
27 79581 K&

In [43]:
db_company_df = pd.read_sql_table(config['DB_TABLE_NAME_1'], con=conn, index_col='id', 
                          columns=['name'])
db_company_df.head()

,name
id,
1,롯데케미칼
2,대한유화
3,이화산업
4,디케이앤디
5,송원산업


In [45]:
company_series = db_company_df['name'].drop_duplicates().reset_index(drop=True)
company_series.head()

0    롯데케미칼
1     대한유화
2     이화산업
3    디케이앤디
4     송원산업
Name: name, dtype: object